In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('SBIN.NS.csv')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
df.iloc[:,1:2]

,Open
0,18.691147
1,18.894005
2,18.327892
3,17.502312
4,17.738192
...,...
6768,599.099976
6769,601.000000
6770,602.099976
6771,598.150024


In [6]:
from tensorflow.keras.models import load_model

In [7]:
model = load_model('sbi_model.h5')

2022-11-29 17:11:03.764103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6773 entries, 0 to 6772
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       6773 non-null   object 
 1   Open       6764 non-null   float64
 2   High       6764 non-null   float64
 3   Low        6764 non-null   float64
 4   Close      6764 non-null   float64
 5   Adj Close  6764 non-null   float64
 6   Volume     6764 non-null   float64
dtypes: float64(6), object(1)
memory usage: 370.5+ KB


(6773, 7)